In [1]:
# Libraries

import geemap, ee # Import geemap and earth engine together
from IPython.display import Image, display

print("Libraries imported")

# Authenticate Google Earth Engine and initialize project

ee.Authenticate() # Authenticate Google Earth Engine account
ee.Initialize(project="geog-581-483717") # Initialize GEOG 581 project

print("Authenticated and initialized")

/Users/marcel/Desktop/geog581-gee-notebook/venv/lib/python3.10/site-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.4) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


Libraries imported
Authenticated and initialized


In [2]:
print('Hello world!')


Hello world!


In [3]:
# Load an image
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')

# Create the interactive map
Map = geemap.Map(ee_initialize=False)

# Center the map on the image
Map.centerObject(image, 9)

# Add layer
Map.addLayer(image)

In [ ]:
# Load an image
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')

# Define visualization parameters in an object literal.
viz_params = {
    "bands": ['B5', 'B4', 'B3'], 
    "min": 0.02, 
    "max": 0.4, 
    "gamma": 1.3}

# Center and add layer
Map.centerObject(image, 9)
Map.addLayer(image, viz_params, 'Landsat 8 false color')

In [5]:
# Adding features and feature collections
counties = ee.FeatureCollection('TIGER/2018/Counties')
Map.addLayer(counties, {}, 'counties');

In [6]:
# Adding image collections
collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')


In [7]:
# Construct a point
point = ee.Geometry.Point(-122.262, 37.8719)

In [8]:
# Define start and end dates
start = ee.Date('2014-06-01')
finish = ee.Date('2014-10-01')

In [9]:
# Filter Landsat 8 image collection
filtered_collection = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")
      .filterBounds(point)
      .filterDate(start, finish)
      .sort("CLOUD_COVER", True)
)


In [10]:
# Get first image with lowest cloud cover
first = filtered_collection.first()

In [11]:
# Filter a feature collection

# Load
feature_collection = ee.FeatureCollection('TIGER/2016/States')

# Filter
filtered_fc = feature_collection.filter(ee.Filter.eq('NAME', 'California'))

# Add layer
Map.setCenter(-119.604, 37.798, 6)
Map.addLayer(filtered_fc, {}, 'California')

In [12]:
# Show map
Map

Map(center=[37.798, -119.604], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [ ]:
# Band Math

# This function gets NDVI from Landsat 5 imagery
def getNDVI(image):
    return image.normalizedDifference(['B4', 'B3'])

# Load two Landsat 5 images, 20 years apart
image1 = ee.Image('LANDSAT/LT05/C02/T1_TOA/LT05_044034_19900604')

image2 = ee.Image('LANDSAT/LT05/C02/T1_TOA/LT05_044034_20100611')

# Compute NDVI from the scenes
ndvi1 = getNDVI(image1)
ndvi2 = getNDVI(image2)

# Compute difference in NDVI
ndvi_difference = ndvi2.subtract(ndvi1)

In [ ]:
# Iterating over items in a collection

# This function gets NDVI from Landsat 8 imagery.
def addNDVI(image):
  return image.addBands(image.normalizedDifference(['B5', 'B4']))

# Load the Landsat 8 TOA data, filter by location and date.
collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA'). filterBounds(ee.Geometry.Point(-122.262, 37.8719)).filterDate('2014-06-01', '2014-10-01')

#  Map the function over the collection.
ndvi_collection = collection.map(addNDVI)

In [ ]:
# Add new property/attribute/field to a feature collection
# This function creates a new property that is the sum of two existing properties.
def addField(feature):
  sum = ee.Number(feature.get('property1')).add(feature.get('property2'))
  return feature.set({'sum': sum})

# Create a FeatureCollection from a list of Features.
features = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point(-122.4536, 37.7403),
    {"property1": 100, "property2": 100}),
    ee.Feature(ee.Geometry.Point(-118.2294, 34.039),
    {"property1": 200, "property2": 300}),
])

# Map the function over the collection.
feature_collection = features.map(addField)

# Print a selected property of one Feature.
print(feature_collection.first().get('sum'))

# Print the entire FeatureCollection.
print(feature_collection)

ee.ComputedObject({
  "functionInvocationValue": {
    "functionName": "Element.get",
    "arguments": {
      "object": {
        "functionInvocationValue": {
          "functionName": "Collection.first",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "Collection.map",
                "arguments": {
                  "baseAlgorithm": {
                    "functionDefinitionValue": {
                      "argumentNames": [
                        "_MAPPING_VAR_0_0"
                      ],
                      "body": {
                        "functionInvocationValue": {
                          "functionName": "Element.set",
                          "arguments": {
                            "key": {
                              "constantValue": "sum"
                            },
                            "object": {
                              "argumentReference": "_MAPPING_VAR_0_0"
        

In [27]:
# Establishing AOI centroid for all images in a collection

# This function returns the image centroid as a new Feature.
def getGeom(image):
  return ee.Feature(image.geometry().centroid(), {"foo": 1})

# Load a Landsat 8 TOA collection.
collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA').filterBounds(ee.Geometry.Point(-122.262, 37.8719)).filterDate('2014-06-01', '2014-10-01');

# Map the function over the ImageCollection
feature_collection = ee.FeatureCollection(collection.map(getGeom))

# Print the collection
print(feature_collection)

ee.FeatureCollection({
  "functionInvocationValue": {
    "functionName": "Collection.map",
    "arguments": {
      "baseAlgorithm": {
        "functionDefinitionValue": {
          "argumentNames": [
            "_MAPPING_VAR_0_0"
          ],
          "body": {
            "functionInvocationValue": {
              "functionName": "Feature",
              "arguments": {
                "geometry": {
                  "functionInvocationValue": {
                    "functionName": "Geometry.centroid",
                    "arguments": {
                      "geometry": {
                        "functionInvocationValue": {
                          "functionName": "Image.geometry",
                          "arguments": {
                            "feature": {
                              "argumentReference": "_MAPPING_VAR_0_0"
                            }
                          }
                        }
                      }
                    }
                  }
   

In [ ]:
# Reduce region

# Load and display a Landsat TOA image
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')
Map.addLayer(image, {"bands": ['B4', 'B3', 'B2'], max: 0.3})

# Create an arbitrary rectangle as a region
region = ee.Geometry.Rectangle(-122.2806, 37.1209, -122.0554, 37.2413)
Map.addLayer(region)

# Get a dictionary of means in the region. Keys are bandnames
mean = image.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=region,
    scale=30
)

# Display region
Map


Map(bottom=47058.0, center=[45.75602615586017, -121.77520751953125], controls=(WidgetControl(options=['positio…

In [40]:
# Masking

# This function gets NDVI from Landsat 5 imagery
def getNDVI(image):
    return image.normalizedDifference(['B4', 'B3'])

# Load two Landsat 5 images, 20 years apart
image1 = ee.Image('LANDSAT/LT05/C02/T1_TOA/LT05_044034_19900604');
image2 = ee.Image('LANDSAT/LT05/C02/T1_TOA/LT05_044034_20100611')

# Compute NDVI from the scenes
ndvi1 = getNDVI(image1)
ndvi2 = getNDVI(image2)

# Compute the difference in NDVI
ndvi_difference = ndvi2.subtract(ndvi1)

# Load the land mask from the SRTM DEM
land_mask = ee.Image('CGIAR/SRTM90_V4').mask()

# Update the NDVI difference mask with the land mask
masked_difference = ndvi_difference.updateMask(land_mask)

# Display the masked result
viz_params = {min: -0.5, max: 0.5, "palette": ['FF0000', 'FFFFFF', '0000FF']}
Map.setCenter(-122.2531, 37.6295, 9)
Map.addLayer(masked_difference, viz_params, 'NDVI difference')
Map

Map(bottom=202373.0, center=[37.6295, -122.2531], controls=(WidgetControl(options=['position', 'transparent_bg…